In [31]:
%load_ext autoreload
%autoreload 2
%pylab inline

using dark_emulator at  /Users/sunao-mac/Documents/python_package/dark_emulator_public/dark_emulator/__init__.py
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Populating the interactive namespace from numpy and matplotlib


In [32]:
import numpy as np
from dark_emulator_public import dark_emulator
import os, sys, time, json
import matplotlib.pyplot as plt
from collections import OrderedDict as od
from scipy.interpolate import InterpolatedUnivariateSpline as ius
from scipy.interpolate import interp2d, interp1d
from scipy.integrate import simps
from tqdm import tqdm
import hsc3x2pt

## Preparation for Fisher
Computation of $C(l)$ takes ~20 sec for each cosmology.
So this notebook pre-compute $C(l)$ for cosmologies needed for Fisher analysis.

Model parameters of 3x2pt analysis with single(double) source redshift bin are
- comsology: 5
- galaxy bias: 1x3
- magnification bias: 1x3
- photo-z: 1x1(2)
- multiplicative bais: 1x1(2)

13(15) parameters in total.

$C(l)$s depend on cosmological parameters, galaxy bias and magnification bias, for which we need to compute $C(l)$.

**Note** : Fisher analysis is doable even if the computational time for one model reaches ~20 sec. However, in the real analysis, ~20 sec is too much to run sampling code. 
I just want to know how people in cosmic shear, like Hikage-san and Hamana-san, or people in 3x2pt, like DES or KiDS, implement code of $C(l)$.?

In [33]:
power_b1 = hsc3x2pt.power_b1_class()

initialize cosmo_class
Initialize pklin emulator
initialize propagator emulator
Initialize sigma_d emulator
initialize cross-correlation emulator
initialize auto-correlation emulator
Initialize sigmaM emulator
initialize xinl emulator


In [34]:
Omega_s_HSC = 140

### List of models to compute $C(l)$ to compute **single** source bin anlaysis
0. fiducial = Planck cosmology + fiducial galaxy bias, magnification bais + 0 photo-z and multiplicative bias
1. $\omega_b h^2$ +
2. $\omega_c h^2$ +
3. $\Omega_{\rm de}$ +
4. $n_{\rm s}$ +
5. $\ln 10^{10}A_{\rm s}$ +
6. $b_{\rm 1,1}$ + 
7. $b_{\rm 1,2}$ + 
8. $b_{\rm 1,3}$ + 
9. $\alpha_{\rm mag,1}$ + 
10. $\alpha_{\rm mag,2}$ + 
11. $\alpha_{\rm mag,3}$ + 
12. $\Delta z_{\rm ph}$ +
13. $\Delta m$ + = easy. Just multiply $1+\Delta m$ to fiducial.

In [35]:
dirname = 'single_source_ClY1'

def compute(params, dir_to_save, compute_lens_cross=False):
    t0 = time.time()
    ombh2, omch2, Ode, sigma8, ns, b11, b12, b13, a1, a2, a3, dzph, dm = params
    g_l1 = hsc3x2pt.galaxy_sample_lens_class(['lowz'  , 0.15, 0.35, b11, '40.36 arcmin^-2', a1])
    g_l2 = hsc3x2pt.galaxy_sample_lens_class(['cmass1', 0.47, 0.55, b12, '20.06 arcmin^-2', a2])
    g_l3 = hsc3x2pt.galaxy_sample_lens_class(['cmass2', 0.55, 0.70, b13, '30.12 arcmin^-2', a3])
    g_s12= hsc3x2pt.galaxy_sample_source_class(['s12', 'sourcePzs_Y1/MLZs12.txt', dzph, dm, 0.2, '8.74 arcmin^-2'])
    pk2cl = hsc3x2pt.pk2cl_class(power_b1)
    pk2cl.set_galaxy_sample(g_l1)
    pk2cl.set_galaxy_sample(g_l2)
    pk2cl.set_galaxy_sample(g_l3)
    pk2cl.set_galaxy_sample(g_s12)
    # set cosmo
    cosmo_dict = dict(zip(['omega_b', 'omega_c', 'Omega_de', 'ln10p10As', 'n_s', 'w_de'], [ombh2, omch2, Ode, 3.094, ns, -1]))
    pk2cl.set_cosmology_from_dict(cosmo_dict)
    sigma8_temp = pk2cl.pk_class.get_sigma8()
    ln10p10As = 3.094 + 2*np.log(sigma8/sigma8_temp)
    print(f'ln10p10As={ln10p10As}')
    cosmo_dict = dict(zip(['omega_b', 'omega_c', 'Omega_de', 'ln10p10As', 'n_s', 'w_de'], [ombh2, omch2, Ode, ln10p10As, ns, -1]))
    pk2cl.set_cosmology_from_dict(cosmo_dict)
    
    pk2cl.init_pk()
    pk2cl.set_Omega_s({'w':8300, 'gamma_t':Omega_s_HSC, 'xi':Omega_s_HSC}) # HSCY1

    l = np.logspace(-2, 5, 1000)
    with hsc3x2pt.Time():
        pk2cl.compute_all_Cl(l, compute_lens_cross=compute_lens_cross)
    pk2cl.dump_Cl_cache(f'{dirname}/{dir_to_save}', overwrite=True)
    t1 = time.time()
    print(f'{t1-t0} sec')

In [36]:
dp = 0.01
file_param = od()

def save_dparam(dparam, dir_to_save):
    fname = os.path.join(dirname, 'file_param.json')
    if os.path.exists(fname):
        file_param = json.load(open(fname, 'r'), object_pairs_hook=od) 
    else:
        file_param = od()
    file_param[dir_to_save] = dparam
    json.dump(file_param, open(fname, 'w'), indent=2)

In [37]:
param_fiducial = [0.02225, 0.1198, 0.6844, 0.831, 0.9645, 1.78, 2.10, 2.28, 2.259, 3.563, 3.729, 0.0, 0.0]
compute(param_fiducial, 'fiducial', compute_lens_cross=True)

ln10p10As=3.0972126913059492
:22.047406911849976 sec
saving l to single_source_ClY1/fiducial/l.txt
saving lowz,lowz to single_source_ClY1/fiducial/lowz,lowz.txt
saving lowz,cmass1 to single_source_ClY1/fiducial/lowz,cmass1.txt
saving lowz,cmass2 to single_source_ClY1/fiducial/lowz,cmass2.txt
saving cmass1,lowz to single_source_ClY1/fiducial/cmass1,lowz.txt
saving cmass1,cmass1 to single_source_ClY1/fiducial/cmass1,cmass1.txt
saving cmass1,cmass2 to single_source_ClY1/fiducial/cmass1,cmass2.txt
saving cmass2,lowz to single_source_ClY1/fiducial/cmass2,lowz.txt
saving cmass2,cmass1 to single_source_ClY1/fiducial/cmass2,cmass1.txt
saving cmass2,cmass2 to single_source_ClY1/fiducial/cmass2,cmass2.txt
saving lowz,s12 to single_source_ClY1/fiducial/lowz,s12.txt
saving cmass1,s12 to single_source_ClY1/fiducial/cmass1,s12.txt
saving cmass2,s12 to single_source_ClY1/fiducial/cmass2,s12.txt
saving s12,s12 to single_source_ClY1/fiducial/s12,s12.txt
saving cosmo_dict to single_source_ClY1/fiducial/

In [38]:
param = [0.02225*(1+dp), 0.1198, 0.6844, 0.831, 0.9645, 1.78, 2.10, 2.28, 2.259, 3.563, 3.729, 0.0, 0.0]
compute(param, 'omega_b')
save_dparam(0.02225*dp, 'omega_b')

ln10p10As=3.1004205154513964
:12.44694709777832 sec
saving l to single_source_ClY1/omega_b/l.txt
saving lowz,lowz to single_source_ClY1/omega_b/lowz,lowz.txt
saving cmass1,cmass1 to single_source_ClY1/omega_b/cmass1,cmass1.txt
saving cmass2,cmass2 to single_source_ClY1/omega_b/cmass2,cmass2.txt
saving lowz,s12 to single_source_ClY1/omega_b/lowz,s12.txt
saving cmass1,s12 to single_source_ClY1/omega_b/cmass1,s12.txt
saving cmass2,s12 to single_source_ClY1/omega_b/cmass2,s12.txt
saving s12,s12 to single_source_ClY1/omega_b/s12,s12.txt
saving cosmo_dict to single_source_ClY1/omega_b/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s12 to galaxy_sample_s12.json
12.814717769622803 sec


In [39]:
param = [0.02225, 0.1198*(1+dp), 0.6844, 0.831, 0.9645, 1.78, 2.10, 2.28, 2.259, 3.563, 3.729, 0.0, 0.0]
compute(param, 'omega_c')
save_dparam(0.1198*dp, 'omega_c')

ln10p10As=3.082645380931363
:11.981634140014648 sec
saving l to single_source_ClY1/omega_c/l.txt
saving lowz,lowz to single_source_ClY1/omega_c/lowz,lowz.txt
saving cmass1,cmass1 to single_source_ClY1/omega_c/cmass1,cmass1.txt
saving cmass2,cmass2 to single_source_ClY1/omega_c/cmass2,cmass2.txt
saving lowz,s12 to single_source_ClY1/omega_c/lowz,s12.txt
saving cmass1,s12 to single_source_ClY1/omega_c/cmass1,s12.txt
saving cmass2,s12 to single_source_ClY1/omega_c/cmass2,s12.txt
saving s12,s12 to single_source_ClY1/omega_c/s12,s12.txt
saving cosmo_dict to single_source_ClY1/omega_c/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s12 to galaxy_sample_s12.json
12.321738958358765 sec


In [40]:
param = [0.02225, 0.1198, 0.6844*(1+dp), 0.831, 0.9645, 1.78, 2.10, 2.28, 2.259, 3.563, 3.729, 0.0, 0.0]
compute(param, 'Omega_de')
save_dparam(0.6844*dp, 'Omega_de')

ln10p10As=3.092196129211238
:12.075910806655884 sec
saving l to single_source_ClY1/Omega_de/l.txt
saving lowz,lowz to single_source_ClY1/Omega_de/lowz,lowz.txt
saving cmass1,cmass1 to single_source_ClY1/Omega_de/cmass1,cmass1.txt
saving cmass2,cmass2 to single_source_ClY1/Omega_de/cmass2,cmass2.txt
saving lowz,s12 to single_source_ClY1/Omega_de/lowz,s12.txt
saving cmass1,s12 to single_source_ClY1/Omega_de/cmass1,s12.txt
saving cmass2,s12 to single_source_ClY1/Omega_de/cmass2,s12.txt
saving s12,s12 to single_source_ClY1/Omega_de/s12,s12.txt
saving cosmo_dict to single_source_ClY1/Omega_de/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s12 to galaxy_sample_s12.json
12.386327028274536 sec


In [41]:
param = [0.02225, 0.1198, 0.6844, 0.831*(1.0+dp), 0.9645, 1.78, 2.10, 2.28, 2.259, 3.563, 3.729, 0.0, 0.0]
compute(param, 'sigma8')
save_dparam(0.831*dp, 'sigma8')

ln10p10As=3.117113353012285
:12.211494207382202 sec
saving l to single_source_ClY1/sigma8/l.txt
saving lowz,lowz to single_source_ClY1/sigma8/lowz,lowz.txt
saving cmass1,cmass1 to single_source_ClY1/sigma8/cmass1,cmass1.txt
saving cmass2,cmass2 to single_source_ClY1/sigma8/cmass2,cmass2.txt
saving lowz,s12 to single_source_ClY1/sigma8/lowz,s12.txt
saving cmass1,s12 to single_source_ClY1/sigma8/cmass1,s12.txt
saving cmass2,s12 to single_source_ClY1/sigma8/cmass2,s12.txt
saving s12,s12 to single_source_ClY1/sigma8/s12,s12.txt
saving cosmo_dict to single_source_ClY1/sigma8/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s12 to galaxy_sample_s12.json
12.53929615020752 sec


In [42]:
param = [0.02225, 0.1198, 0.6844, 0.831, 0.9645*(1.0+dp), 1.78, 2.10, 2.28, 2.259, 3.563, 3.729, 0.0, 0.0]
compute(param, 'ns')
save_dparam(0.9645*dp, 'ns')

ln10p10As=3.0899364412116435
:11.95534896850586 sec
saving l to single_source_ClY1/ns/l.txt
saving lowz,lowz to single_source_ClY1/ns/lowz,lowz.txt
saving cmass1,cmass1 to single_source_ClY1/ns/cmass1,cmass1.txt
saving cmass2,cmass2 to single_source_ClY1/ns/cmass2,cmass2.txt
saving lowz,s12 to single_source_ClY1/ns/lowz,s12.txt
saving cmass1,s12 to single_source_ClY1/ns/cmass1,s12.txt
saving cmass2,s12 to single_source_ClY1/ns/cmass2,s12.txt
saving s12,s12 to single_source_ClY1/ns/s12,s12.txt
saving cosmo_dict to single_source_ClY1/ns/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s12 to galaxy_sample_s12.json
12.2799711227417 sec


In [43]:
param = [0.02225, 0.1198, 0.6844, 0.831, 0.9645, 1.78*(1.0+dp), 2.10, 2.28, 2.259, 3.563, 3.729, 0.0, 0.0]
compute(param, 'b1lowz')
save_dparam(1.78*dp, 'b1lowz')

ln10p10As=3.0972126913059492
:11.966985940933228 sec
saving l to single_source_ClY1/b1lowz/l.txt
saving lowz,lowz to single_source_ClY1/b1lowz/lowz,lowz.txt
saving cmass1,cmass1 to single_source_ClY1/b1lowz/cmass1,cmass1.txt
saving cmass2,cmass2 to single_source_ClY1/b1lowz/cmass2,cmass2.txt
saving lowz,s12 to single_source_ClY1/b1lowz/lowz,s12.txt
saving cmass1,s12 to single_source_ClY1/b1lowz/cmass1,s12.txt
saving cmass2,s12 to single_source_ClY1/b1lowz/cmass2,s12.txt
saving s12,s12 to single_source_ClY1/b1lowz/s12,s12.txt
saving cosmo_dict to single_source_ClY1/b1lowz/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s12 to galaxy_sample_s12.json
12.317566871643066 sec


In [44]:
param = [0.02225, 0.1198, 0.6844, 0.831, 0.9645, 1.78, 2.10*(1.0+dp), 2.28, 2.259, 3.563, 3.729, 0.0, 0.0]
compute(param, 'b1cmass1')
save_dparam(2.10*dp, 'b1cmass1')

ln10p10As=3.0972126913059492
:13.598771810531616 sec
saving l to single_source_ClY1/b1cmass1/l.txt
saving lowz,lowz to single_source_ClY1/b1cmass1/lowz,lowz.txt
saving cmass1,cmass1 to single_source_ClY1/b1cmass1/cmass1,cmass1.txt
saving cmass2,cmass2 to single_source_ClY1/b1cmass1/cmass2,cmass2.txt
saving lowz,s12 to single_source_ClY1/b1cmass1/lowz,s12.txt
saving cmass1,s12 to single_source_ClY1/b1cmass1/cmass1,s12.txt
saving cmass2,s12 to single_source_ClY1/b1cmass1/cmass2,s12.txt
saving s12,s12 to single_source_ClY1/b1cmass1/s12,s12.txt
saving cosmo_dict to single_source_ClY1/b1cmass1/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s12 to galaxy_sample_s12.json
13.969033002853394 sec


In [45]:
param = [0.02225, 0.1198, 0.6844, 0.831, 0.9645, 1.78, 2.10, 2.28*(1.0+dp), 2.259, 3.563, 3.729, 0.0, 0.0]
compute(param, 'b1cmass2')
save_dparam(2.28*dp, 'b1cmass2')

ln10p10As=3.0972126913059492
:12.548655986785889 sec
saving l to single_source_ClY1/b1cmass2/l.txt
saving lowz,lowz to single_source_ClY1/b1cmass2/lowz,lowz.txt
saving cmass1,cmass1 to single_source_ClY1/b1cmass2/cmass1,cmass1.txt
saving cmass2,cmass2 to single_source_ClY1/b1cmass2/cmass2,cmass2.txt
saving lowz,s12 to single_source_ClY1/b1cmass2/lowz,s12.txt
saving cmass1,s12 to single_source_ClY1/b1cmass2/cmass1,s12.txt
saving cmass2,s12 to single_source_ClY1/b1cmass2/cmass2,s12.txt
saving s12,s12 to single_source_ClY1/b1cmass2/s12,s12.txt
saving cosmo_dict to single_source_ClY1/b1cmass2/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s12 to galaxy_sample_s12.json
12.886399030685425 sec


In [46]:
param = [0.02225, 0.1198, 0.6844, 0.831, 0.9645, 1.78, 2.10, 2.28, 2.259*(1.0+dp), 3.563, 3.729, 0.0, 0.0]
compute(param, 'alphamaglowz')
save_dparam(2.259*dp, 'alphamaglowz')

ln10p10As=3.0972126913059492
:12.651279926300049 sec
saving l to single_source_ClY1/alphamaglowz/l.txt
saving lowz,lowz to single_source_ClY1/alphamaglowz/lowz,lowz.txt
saving cmass1,cmass1 to single_source_ClY1/alphamaglowz/cmass1,cmass1.txt
saving cmass2,cmass2 to single_source_ClY1/alphamaglowz/cmass2,cmass2.txt
saving lowz,s12 to single_source_ClY1/alphamaglowz/lowz,s12.txt
saving cmass1,s12 to single_source_ClY1/alphamaglowz/cmass1,s12.txt
saving cmass2,s12 to single_source_ClY1/alphamaglowz/cmass2,s12.txt
saving s12,s12 to single_source_ClY1/alphamaglowz/s12,s12.txt
saving cosmo_dict to single_source_ClY1/alphamaglowz/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s12 to galaxy_sample_s12.json
12.991662740707397 sec


In [47]:
param = [0.02225, 0.1198, 0.6844, 0.831, 0.9645, 1.78, 2.10, 2.28, 2.259, 3.563*(1.0+dp), 3.729, 0.0, 0.0]
compute(param, 'alphamagcmass1')
save_dparam(3.563*dp, 'alphamagcmass1')

ln10p10As=3.0972126913059492
:12.164007902145386 sec
saving l to single_source_ClY1/alphamagcmass1/l.txt
saving lowz,lowz to single_source_ClY1/alphamagcmass1/lowz,lowz.txt
saving cmass1,cmass1 to single_source_ClY1/alphamagcmass1/cmass1,cmass1.txt
saving cmass2,cmass2 to single_source_ClY1/alphamagcmass1/cmass2,cmass2.txt
saving lowz,s12 to single_source_ClY1/alphamagcmass1/lowz,s12.txt
saving cmass1,s12 to single_source_ClY1/alphamagcmass1/cmass1,s12.txt
saving cmass2,s12 to single_source_ClY1/alphamagcmass1/cmass2,s12.txt
saving s12,s12 to single_source_ClY1/alphamagcmass1/s12,s12.txt
saving cosmo_dict to single_source_ClY1/alphamagcmass1/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s12 to galaxy_sample_s12.json
12.494340181350708 sec


In [48]:
param = [0.02225, 0.1198, 0.6844, 0.831, 0.9645, 1.78, 2.10, 2.28, 2.259, 3.563, 3.729*(1.0+dp), 0.0, 0.0]
compute(param, 'alphamagcmass2')
save_dparam(3.729*dp, 'alphamagcmass2')

ln10p10As=3.0972126913059492
:12.649302005767822 sec
saving l to single_source_ClY1/alphamagcmass2/l.txt
saving lowz,lowz to single_source_ClY1/alphamagcmass2/lowz,lowz.txt
saving cmass1,cmass1 to single_source_ClY1/alphamagcmass2/cmass1,cmass1.txt
saving cmass2,cmass2 to single_source_ClY1/alphamagcmass2/cmass2,cmass2.txt
saving lowz,s12 to single_source_ClY1/alphamagcmass2/lowz,s12.txt
saving cmass1,s12 to single_source_ClY1/alphamagcmass2/cmass1,s12.txt
saving cmass2,s12 to single_source_ClY1/alphamagcmass2/cmass2,s12.txt
saving s12,s12 to single_source_ClY1/alphamagcmass2/s12,s12.txt
saving cosmo_dict to single_source_ClY1/alphamagcmass2/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s12 to galaxy_sample_s12.json
12.963857173919678 sec


In [49]:
param = [0.02225, 0.1198, 0.6844, 0.831, 0.9645, 1.78, 2.10, 2.28, 2.259, 3.563, 3.729, dp, 0.0]
compute(param, 'dzph')
save_dparam(dp, 'dzph')

ln10p10As=3.0972126913059492
:12.490612983703613 sec
saving l to single_source_ClY1/dzph/l.txt
saving lowz,lowz to single_source_ClY1/dzph/lowz,lowz.txt
saving cmass1,cmass1 to single_source_ClY1/dzph/cmass1,cmass1.txt
saving cmass2,cmass2 to single_source_ClY1/dzph/cmass2,cmass2.txt
saving lowz,s12 to single_source_ClY1/dzph/lowz,s12.txt
saving cmass1,s12 to single_source_ClY1/dzph/cmass1,s12.txt
saving cmass2,s12 to single_source_ClY1/dzph/cmass2,s12.txt
saving s12,s12 to single_source_ClY1/dzph/s12,s12.txt
saving cosmo_dict to single_source_ClY1/dzph/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s12 to galaxy_sample_s12.json
12.803786993026733 sec


In [50]:
param = [0.02225, 0.1198, 0.6844, 0.831, 0.9645, 1.78, 2.10, 2.28, 2.259, 3.563, 3.729, 0.0, dp]
compute(param, 'dm')
save_dparam(dp, 'dm')

ln10p10As=3.0972126913059492
:11.887729167938232 sec
saving l to single_source_ClY1/dm/l.txt
saving lowz,lowz to single_source_ClY1/dm/lowz,lowz.txt
saving cmass1,cmass1 to single_source_ClY1/dm/cmass1,cmass1.txt
saving cmass2,cmass2 to single_source_ClY1/dm/cmass2,cmass2.txt
saving lowz,s12 to single_source_ClY1/dm/lowz,s12.txt
saving cmass1,s12 to single_source_ClY1/dm/cmass1,s12.txt
saving cmass2,s12 to single_source_ClY1/dm/cmass2,s12.txt
saving s12,s12 to single_source_ClY1/dm/s12,s12.txt
saving cosmo_dict to single_source_ClY1/dm/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s12 to galaxy_sample_s12.json
12.20041298866272 sec


In [51]:
np.savetxt(os.path.join(dirname, 'param_fiducial.txt'), np.array(param_fiducial))